This notebook tries to resolve the stackoverflow http://stackoverflow.com/questions/17952873/filter-id-range-in-elasticsearch question with an adhoc example

In [5]:
import json
from elasticsearch import Elasticsearch

#cluster node to query
es = Elasticsearch(['localhost:9200',])
index_name    = 'country_currency'

es.indices.delete(
    index = index_name,
)
es.indices.create(
    index = index_name,
)

records = [
    {'country': 1, 'currency': 1, 'id': 'p01'},
    {'country': 1, 'currency': 2, 'id': 'p02'},
    {'country': 1, 'currency': 3, 'id': 'p03'},
    {'country': 2, 'currency': 1, 'id': 'p04'},
    {'country': 2, 'currency': 2, 'id': 'p05'},
    {'country': 2, 'currency': 4, 'id': 'p06'},
    {'country': 3, 'currency': 1, 'id': 'p07'},
    {'country': 3, 'currency': 3, 'id': 'p08'},
    {'country': 3, 'currency': 4, 'id': 'p09'},
    {'country': 4, 'currency': 5, 'id': 'p10'},
    {'country': 4, 'currency': 6, 'id': 'p11'},
    {'country': 4, 'currency': 7, 'id': 'p12'}
]
    
kwargs = {
    'body' : []
}
index_config = { 
    "_type"  : "item", 
    "_index" : 'country_currency', 
}

for idx,r in enumerate(records):
    _index_config = dict(index_config)
    #set Elasticsearch uid
    _index_config['_id'] = idx
    #replicate in a document field to be able to filter for
    r['id'] = idx
    kwargs['body'].append({'index' : _index_config})
    kwargs['body'].append(r)

_ = es.bulk(**kwargs)

In [14]:
elasticsearch_query = {
    "query": {
        "filtered": {
            "filter": {
                "range": {
                    "id": {
                        "gte" : 3,
                        "lt"  : 5
                    }
                }
            }
        }
    }
}

es_kwargs = { 
    "doc_type"  : "item", 
    "index" : "country_currency", 
    "body" : elasticsearch_query,
}


res = es.search(**es_kwargs)

for hit in res.get('hits',{}).get('hits',[]):
    print  hit['_source']
    

{u'country': 2, u'id': 4, u'currency': 2}
{u'country': 2, u'id': 3, u'currency': 1}
